In [154]:
import os
import numpy as np
import pandas as pd
import ujson as json
#import json
import pymongo


### mongodb driver

In [113]:
# settings
# datasets = "2newgroup,imbd,quora"
# k = 5 #20%test
# vector_sizes = [10, 25, 50, 75, 100]
# algorithms_list = ['lr_mvdt', 'rs_mvdt']
# epochs_list = [100, 300, 500, 800, 1000]
# n_features_list = [2, 5, 10, 20, all]  # should below min vector_size
# depth_list = [3, 6, 9, 12, 15]
# min_leaf_point_list = [5, 10, 15, 20, 30]

datasets = "20ng_CG_RM,20newsgroup"#"imdb"
k = 2
vector_sizes = [10, 25]
algorithms_list = ['lr_mvdt', 'rs_mvdt']
epochs_list = [50, 100]
n_features_list = [2, all]
min_leaf_point_list = [5, 10]

In [48]:
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client.thesis

In [49]:
db.list_collection_names()

['result']

In [ ]:
db.result.find_one()

In [103]:
def read_data(dataLocation):
    """
   Reading data and returning in proper format
   :param dataLocation: location of data
   :return: set of features, labels and combine
   """
    df = pd.read_csv(dataLocation)

    X = np.array(df[df.columns[:-1]].values.tolist(), dtype=np.float64)
    y = np.array(df[df.columns[-1]].values.tolist())
    return [X, y]


In [159]:
def avg_by_n_features_lrrs():
    for name in datasets.split(","):
        for n in range(1, k+1):        
            count = str(n)
            for vector_size in vector_sizes:
                for algorithm in algorithms_list:                    
                    get_average_by_n_features(name, n, vector_size, algorithm)
                    for epochs in epochs_list:                                                                   
                        get_average_by_n_features(name, n, vector_size, algorithm, epochs)
                        for min_leaf_point in min_leaf_point_list:                            
                            get_average_by_n_features(name, n, vector_size, algorithm, epochs, min_leaf_point)
                            for n_features in n_features_list: 
                                if n_features == all:                                
                                    data_loc = "data/"+name+"/k"+count+"/final/"
                                    data_train = str(vector_size) + "D_" + name + "_k" + count + "_train.csv"                                

                                    train = read_data(data_loc + data_train)                                
                                    n_features = len(train[0][0])

                                total += 1        
                                #print(name, n, vector_size, algorithm, epochs, min_leaf_point,n_features)
                                get_average_by_n_features(name, n, vector_size, algorithm, epochs, min_leaf_point,n_features)
    print(total)
    
# def avg_by_n_features_cart():
#     for name in datasets.split(","):
#         for n in range(1, k+1):        
#             count = str(n)
#             for vector_size in vector_sizes:                                                                                                             
#                 for min_leaf_point in min_leaf_point_list:                            
#                     for n_features in n_features_list: 
#                         total += 1        
#                         #print(name, n, vector_size, algorithm, epochs, min_leaf_point,n_features)
#                         get_average_by_n_features(name, n, vector_size, algorithm, epochs, min_leaf_point,n_features)                
#     print(total)

In [189]:
# Python program to illustrate 
# *args for variable number of arguments 
def myFun(*argv): 
    print(len(argv))
    print(argv[-1])
    
myFun('Hello', 'Welcome', 'to', 'GeeksforGeeks') 


4
GeeksforGeeks


In [152]:
# argument order
#name, n, vector_size, algorithm, epochs, min_leaf_point,n_features
def get_average_by_n_features(*argv):
    if len(argv) == 7:
        # average by n_features in 10 runs
        query = [{'$match': {'dataset': name, 'k_fold': n, 'd2v_vec_size':vector_size,
                                                 'algorithm':algorithm, 'epochs':epochs, 'feature_size':n_features,
                                                 'min_leaf_point':min_leaf_point 
                                                }
                                     },
                 {'$group': {'_id':'$min_leaf_point', 'count': {'$sum':1},
                            'train_acc': { '$avg': {'$arrayElemAt': [ '$accuracy', 0 ]}},
                            'test_acc': { '$avg': { '$arrayElemAt': ['$accuracy', 1]}},
                             'train_pre': { '$avg': { '$arrayElemAt': ['$precision', 0]}},
                             'test_pre': { '$avg': { '$arrayElemAt': ['$precision', 1]}},
                             'train_rec': { '$avg': { '$arrayElemAt': ['$recall', 0]}},
                             'test_rec': { '$avg': { '$arrayElemAt': ['$recall', 1]}},
                             'train_f1': { '$avg': { '$arrayElemAt': ['$f1', 0]}},
                             'test_f1': { '$avg': { '$arrayElemAt': ['$f1', 1]}},
                            }
                 }]
        #print(query)
        d = db.result.aggregate(query)                                
        for i in d:
            #print(i)
            new_result = {'dataset': name,  # name of dataset                  
                          'k_fold': int(n),  # which K_fold number
                          'd2v_vec_size': int(vector_size),  # dod2vec feature dimension
                          'algorithm': algorithm,  # algorithm name
                          'epochs': epochs,  # no of epochs
                          'min_leaf_point': min_leaf_point,  # given depth of tree
                          'feature_size': n_features,  # how may features are taken                  
                          'accuracy': [train_acc, test_acc],
                          'precision': [train_pre, test_pre], # precision of both train and test                      
                          'recall': [train_rec, test_rec],# recall of both train and test
                          'f1': [train_f1, test_f1]
                         }
            #print(new_result)
            save_results(new_result, "by_n_features")
    elif len(argv) == 6:
        #

In [156]:
def save_results(new_data, isfrom):
    # result file is not exits
    result_file = isfrom+".json"
    if not os.path.exists(result_file):
        temp = {"results": []}
        with open(result_file, 'w+') as json_result:
            json.dump(temp, json_result)

    with open(result_file) as json_file:
        data = json.load(json_file)
        temp = data["results"]
        # python object to be appended
        temp.append(new_data)

    # then update data
    with open(result_file, 'w') as f:
        json.dump(data, f)

In [84]:
for i in d:
    print(i)

{'_id': 7, 'count': 1, 'train_avg_acc': 0.8314014752370916, 'test_avg_acc': 0.6501580611169653}
{'_id': 6, 'count': 1, 'train_avg_acc': 0.8103266596417281, 'test_avg_acc': 0.6090621707060063}
{'_id': 9, 'count': 1, 'train_avg_acc': 0.7987355110642782, 'test_avg_acc': 0.6564805057955743}
{'_id': 4, 'count': 1, 'train_avg_acc': 0.8103266596417281, 'test_avg_acc': 0.6343519494204426}
{'_id': 3, 'count': 1, 'train_avg_acc': 0.7924130663856691, 'test_avg_acc': 0.6195995785036881}
{'_id': 0, 'count': 1, 'train_avg_acc': 0.827186512118019, 'test_avg_acc': 0.6585879873551106}
{'_id': 1, 'count': 1, 'train_avg_acc': 0.8229715489989463, 'test_avg_acc': 0.6649104320337197}
{'_id': 2, 'count': 1, 'train_avg_acc': 0.8356164383561644, 'test_avg_acc': 0.6469968387776607}
{'_id': 5, 'count': 1, 'train_avg_acc': 0.8029504741833509, 'test_avg_acc': 0.6596417281348789}
{'_id': 8, 'count': 1, 'train_avg_acc': 0.8008429926238145, 'test_avg_acc': 0.660695468914647}


In [51]:

query = [{'$match': {'dataset': '20ng_CG_RM', 'k_fold': 1, 'd2v_vec_size':10,
                 'algorithm':'lr_mvdt', 'epochs':50, 'min_leaf_point':5, 'feature_size':2
                    }
         },
        {'$group': {'_id':'$dataset', 'count': {'$sum':1},
                'train_avg_acc': { '$avg': {'$arrayElemAt': [ '$accuracy', 0 ]}},
                'test_avg_acc': { '$avg': { '$arrayElemAt': ['$accuracy', 1]}},
                #'k_fold': {'$push': '$k_fold'}
                   }
        },
        {'$project': {'count':1, 'k_fold':1, 'd2v_vec_size':1, 'train_avg_acc':1, 'test_avg_acc':1
                     }
        }]
d = db.result.aggregate(query)
for i in d:
    print(i)

{'_id': '20ng_CG_RM', 'count': 10, 'train_avg_acc': 0.8388211382113822, 'test_avg_acc': 0.8179695431472082}


In [57]:
i

{'_id': '20ng_CG_RM',
 'count': 10,
 'train_avg_acc': 0.8388211382113822,
 'test_avg_acc': 0.8179695431472082}

In [2]:
with open("results.json") as json_file: 
    data = json.load(json_file)

In [34]:
#get average accuracy by dataset
n = 0
total_train = 0
total_test = 0
dataset = ""
for i, result in enumerate(data["results"]):
    n = i      
    #print(dataset)
    if dataset != result["dataset"]:        
        dataset = result["dataset"]
        #datset = result["dataset"]
        print(dataset)
#    total_train += result["accuracy"][0]
#    total_test += result["accuracy"][1]  

#print(total_train/(n+1))
#print(total_test/(n+1))

20ng_CG_RM
20newsgroup
20ng_CG_RM
20newsgroup


In [35]:
data["results"][0].keys()

dict_keys(['dataset', 'filename', 'k_fold', 'd2v_vec_size', 'algorithm', 'epochs', 'min_leaf_point', 'feature_size', 'd2v_shape', 'run', 'accuracy', 'precision', 'recall', 'f1', 'confusion_matrix', 'max_depth', 'inner_node', 'leaf_node', 'all_node', 'train_true_predict', 'test_true_predict', 'branch_sizes', 'training_time', 'tree_location', 'intermediate_result'])

In [ ]:
lets take average
dataset, algorithm, accuracy

10_run = 10 run average
fet = all feature_size, 10_trun
mlp = min_leaf_point,  fet
ep = epochs, mlp


In [3]:
def get_max_min_depth(data):
    max_depth = 0
    max_min_depth = 1
    for result in data["results"]:
        if result['max_depth'] > max_depth:
            max_depth = result['max_depth']
        if result['max_depth'] < max_min_depth:
            max_min_depth = result['max_depth']
            print(result['max_depth'])        

    return max_depth, max_min_depth

In [4]:
get_max_min_depth(data)

(24, 1)

In [43]:
def get_avg_acc_by(data, filed_name="epochs"):
    pass

In [44]:
n = len(data["results"])

In [45]:
# count based onalgorith, dataset
# then using other hyperparameters
# based on that find average accuracy

In [55]:
datasets = ["20ng_CG_RM","20newsgroup"]
algoriths = ["lr_mvdt", "rs_mvdt", "cart"]

In [ ]:
d = {}
count = 0 
for results in data["results"]:
    for dataset in datasets:        
        if results["dataset"] == dataset:
            print(dataset)
            count += 1
            d[results["dataset"]] = count
        
        #print(results["dataset"])
d

In [5]:
len(data["results"])

144

In [12]:
data["results"][2].keys()

dict_keys(['dataset', 'filename', 'k_fold', 'd2v_vec_size', 'algorithm', 'epochs', 'min_leaf_point', 'feature_size', 'd2v_shape', 'run', 'accuracy', 'precision', 'recall', 'f1', 'confusion_matrix', 'max_depth', 'inner_node', 'leaf_node', 'all_node', 'train_true_predict', 'test_true_predict', 'branch_sizes', 'training_time', 'tree_location', 'intermediate_result'])

In [ ]:
data["results"][2]["intermediate_result"]